In [206]:
import pandas as pd
import numpy as np
from sql_engine import connect
import glob
import traceback
import re

In [207]:
df_2023 = pd.read_excel(r'C:\Medline\8. database\Asia Inspection Database\2022\QP-00017-F-00005 Asia Inspection Database 2023.XLSM',sheet_name="Sheet1")

C:\Users\ypeng\AppData\Local\Temp\ipykernel_16292\1272271608.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_2023 = pd.read_excel(r'C:\Medline\8. database\Asia Inspection Database\2022\QP-00017-F-00005 Asia Inspection Database 2023.XLSM',sheet_name="Sheet1")


In [208]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2023_v1.xlsx')

In [209]:
create_mapping = pd.read_excel(r'C:\Medline\8. database\DIV_map.xlsx',sheet_name=2)

In [210]:
manager_dict_ct = dict(zip(create_mapping['Creasted'],create_mapping['Regional Manager']))
supervisor_dict_ct = dict(zip(create_mapping['Creasted'],create_mapping['Supervisor']))
scheduler_dict_ct = dict(zip(create_mapping['Creasted'],create_mapping['Scheduler']))

In [211]:
manager_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Regional Manager']))
supervisor_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Supervisor']))
scheduler_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Scheduler']))

In [212]:
org_dict = {
    'manager_ct':manager_dict_ct,
    'manager':manager_dict,
    'supervisor_ct':supervisor_dict_ct,
    'supervisor':supervisor_dict,
    'scheduler_ct':scheduler_dict_ct,
    'scheduler':scheduler_dict  
}

In [213]:
def assign_inspection_result(df):
    return df.assign(**{'Inspection Result' : lambda d : d['Results'].map({'A':'Inspected', 'R':'Inspected', 'W':'Waived', 'G':'Guaranteed'})})

In [214]:
# 定义一个函数，输入一个日期，返回那一天所在周的周二是哪一天
def get_last_tuesday_of_week(date):
    # 将输入的日期转换为Timestamp对象
    date = pd.to_datetime(date)
    # 计算输入日期所在周的第一天，即周日的日期
    sunday_date = date - pd.Timedelta(days=date.dayofweek + 1)
    # 计算输入日期所在周的上周二的日期，即周日的日期减五天
    tuesday_date = sunday_date + pd.Timedelta(days=-5)
    # 返回周二的日期，格式为YYYY-MM-DD
    return tuesday_date.strftime('%m%d')

In [215]:
def get_org(row,type):    
    if row["Results"] in ["A", "R"]:
        key = row["Vendor Code"]
        map = org_dict[type]
    else:
        key = row["Created By"]
        map = org_dict[type + "_ct"]
    # 从map中获取key对应的值，如果没有找到，返回None
    return map.get(key)

In [216]:
def extract_div(x):
    # 如果x以ISP开头，直接返回x
    if x.startswith("ISP"):
        return x
    # 否则，使用正则表达式提取数字
    else:
        # 导入re库
        import re
        # 使用re.search()方法找到第一个匹配的数字
        match = re.search("\d+", x)
        # 如果找到了，返回匹配的字符串
        if match:
            return match.group()
        # 否则，返回空值
        else:
            return None

In [217]:
std = 10
df_inspection = (
    df_2023.query('`Inspection Date`.dt.month == @std')
    .pipe(assign_inspection_result)
    .assign(**{
        "Division" : lambda d : d["Division"].apply(extract_div),
        "Supervisor" : lambda d : d.apply(get_org,type = 'supervisor',axis=1),
        "Regional Manager" : lambda d : d.apply(get_org,type = 'manager',axis=1),
        "Scheduler" : lambda d : d.apply(get_org,type = 'scheduler',axis=1),
        'version' : lambda d: d['Inspection Date'].apply(get_last_tuesday_of_week),
        'priority_week' : lambda d : d['Inspection Date'].dt.isocalendar().week -1,
        'week' : lambda d :d['Inspection Date'].dt.strftime('%Y-W%W'),
        'key' : lambda d : d['Vendor'] + d['Item Number'].map(str)
    })
    .sort_values('Inspection Date',ascending=True)
)

In [218]:
df_inspection.shape

(33329, 29)

In [219]:
df_inspection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33329 entries, 99799 to 279695
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  33329 non-null  object        
 1   PO Number           33328 non-null  object        
 2   Lot Number          33303 non-null  object        
 3   Vendor Code         33329 non-null  object        
 4   Vendor              33329 non-null  object        
 5   Factory             3383 non-null   object        
 6   Division            33329 non-null  object        
 7   Inspection Date     33329 non-null  datetime64[ns]
 8   Inspector           11864 non-null  object        
 9   Manufacture Date    33329 non-null  datetime64[ns]
 10  Item Number         33329 non-null  object        
 11  Qty EA              33329 non-null  object        
 12  Results             33329 non-null  object        
 13  Reject Code         99 non-null     objec

In [220]:
# df_inspection.loc[df_inspection['version'] == '0919','version'] = 'aaa'

In [221]:
df_inspection['version'].unique()

array(['0919', '0926', '1003', '1010', '1017', '1024'], dtype=object)

In [222]:
list_prio = list(df_inspection['version'].unique())

In [223]:
priority_path = r'C:\Medline\9. Inspection Priority List\Priority List_input\2023\priority_list'

In [224]:
def combine_priority(path):
    try:
        finalexcelsheet = pd.DataFrame()
        # filenames = glob.glob(path + "\*.xlsx")
        for file in list_prio:
            df = pd.read_excel(path +'\Inspection Priority List 2023' + file + '.xlsx',sheet_name='Priority list',usecols="A,B,C,E,BC")
            df['key'] = df['Vendor Name'] + df['Material Number'].map(str)
            df.drop_duplicates(subset=['key'],keep='first',inplace=True)
            df['version'] = file
            finalexcelsheet = pd.concat([finalexcelsheet,df],ignore_index=True)
        return finalexcelsheet
    except Exception as e:
            traceback.print_exc()

In [225]:
df_priority = combine_priority(priority_path)

In [ ]:
df_InspectionEfficiency_out = df_inspection.merge(df_priority,how='left',on=['version','key'])

In [ ]:
df_InspectionEfficiency_out.to_excel('InspectionEfficiency.xlsx',index = False)

In [8]:
df1 = pd.DataFrame({'B': [1,2], 'D': [100,200]})
df2 = pd.DataFrame({'B': [3,4], 'E': [300,400]})
df3 = pd.DataFrame({'B': [5,6], 'F': [500,600]})

df = pd.DataFrame({'A': ['a', 'a', 'b', 'b', 'c', 'c'], 'B': [1,2,3,4,5,6], 'C': [10,20,30,40,50,60]})

# 先定义一个字典，存储不同的表



In [9]:
tables = {'a': df1, 'b': df2, 'c': df3}

# 定义一个函数，根据A列的值，返回对应的表
def get_table(x):
    return tables[x]

# 使用groupby和apply，对每一组分别调用merge函数，实现left join
result = df.groupby('A').apply(lambda x: pd.merge(x, get_table(x['A'].iloc[0]), on='B', how='left'))

# 重置索引，去掉多余的层级
result = result.reset_index(drop=True)

In [17]:
# 导入pandas库
import pandas as pd

# 创建dataframe
df = pd.DataFrame({"ID": [1, 2, 3, 4, 5, 6], "Code": [123, 345, 456, 567, 789, 432], "Type": ["A", "A", "R", "R", "G", "W"], "Name": ["Jim", "Tom", "Jack", "Jim", "Tony", "Tony"]})

# 创建Code和Name的映射表
code_map = pd.DataFrame({"Code": [123, 345,456], "Scheduler": ["李四", "王五","彭帅"]})
name_map = pd.DataFrame({"Name": ["Jack", "Jim", "Tony"], "Scheduler": ["汤姆", "杰瑞", "林三"]})

# 定义一个函数，根据Type的值来选择合适的key和映射表
def get_scheduler(row):
    if row["Type"] in ["A","R"]:
        key = "Code"
        map = code_map
    else:
        key = "Name"
        map = name_map
    # 用pandas的merge方法来合并dataframe和映射表，返回Scheduler的值
    return pd.merge(df, map, on=key, how="left")["Scheduler"][row.name]

# 用apply方法来对每一行应用函数，得到Scheduler列
df["Scheduler"] = df.apply(get_scheduler, axis=1)

# 打印结果
print(df)


   ID  Code Type  Name Scheduler
0   1   123    A   Jim        李四
1   2   345    A   Tom        王五
2   3   456    R  Jack        彭帅
3   4   567    R   Jim       NaN
4   5   789    G  Tony        林三
5   6   432    W  Tony        林三


In [205]:
# 导入pandas库
import pandas as pd

# 创建一个数据框，包含ID和Division两列
df = pd.DataFrame({"ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], "Division": ["41-Surgeon Glove", "21-Proxima", "81-Primary Care", "40-Exam Glove", "50-Medcrest-Garments", "22-SPT", "30-DME", "51-Preventive care", "82-Personal Care", "ISP-EU", "15-Central Sterile", "29-Anesthesia", "ISP-ANZ"]})

# 定义一个函数，提取Division字段中不以ISP开头的值中的数字，若以ISP开头则保持原来的值
def extract_number(x):
    # 如果x以ISP开头，直接返回x
    if x.startswith("ISP"):
        return x
    # 否则，使用正则表达式提取数字
    else:
        # 导入re库
        import re
        # 使用re.search()方法找到第一个匹配的数字
        match = re.search("\d+", x)
        # 如果找到了，返回匹配的字符串
        if match:
            return match.group()
        # 否则，返回空值
        else:
            return None

# 使用apply()方法对Division列应用extract_number函数，并创建一个新列
df["Number"] = df["Division"].apply(extract_number)

# 打印结果
print(df)


    ID              Division   Number
0    1      41-Surgeon Glove       41
1    2            21-Proxima       21
2    3       81-Primary Care       81
3    4         40-Exam Glove       40
4    5  50-Medcrest-Garments       50
5    6                22-SPT       22
6    7                30-DME       30
7    8    51-Preventive care       51
8    9      82-Personal Care       82
9   10                ISP-EU   ISP-EU
10  11    15-Central Sterile       15
11  12         29-Anesthesia       29
12  13               ISP-ANZ  ISP-ANZ
